In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import os
import copy
import argparse
import numpy as np
import lsst.daf.persistence   as dafPersist
import lsst.afw.coord         as afwCoord
import lsst.afw.image         as afwImage
import lsst.afw.geom          as afwGeom
import lsst.afw.table         as afwTable

# Matplotlib default settings
import matplotlib as mpl 
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = 12, 10
mpl.rcParams['xtick.major.size'] = 8.0
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['xtick.minor.size'] = 4.0
mpl.rcParams['xtick.minor.width'] = 1.5
mpl.rcParams['ytick.major.size'] = 8.0
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['ytick.minor.size'] = 4.0
mpl.rcParams['ytick.minor.width'] = 1.5
mpl.rc('axes', linewidth=2)

import cubehelix  # Cubehelix color scheme from https://github.com/jradavenport/cubehelix
ccmap = cubehelix.cmap(start=0.5, rot=-0.8, minSat=1.2, maxSat=1.2, 
                               minLight=0., maxLight=1., gamma=1.0)

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Shapely related imports
from shapely.geometry import Polygon
from shapely.ops      import cascaded_union
from shapely          import wkb
from descartes        import PolygonPatch

import coaddPatchShape  as coaddPS
#getPolyUnion, polySaveWkb, polyReadWkb 
import coaddPatchNoData as coaddND
#polySaveReg

In [3]:
def wcsGetRaDecPair(wcsInfo, xpos, ypos): 
    
    raRad, decRad = wcsInfo.pixelToSky(xpos, ypos)
    return raRad.asDegrees(), decRad.asDegrees()

In [4]:
def getTractXYDim(tractInfo): 
    
    tractExtent = tractInfo.getBBox().getDimensions()
    return tractExtent.getX(), tractExtent.getY()

In [5]:
def fourCornerRaDec(tractWcs, xDim, yDim): 
    
    xCorners = [1, 1, xDim, xDim]
    yCorners = [1, yDim, yDim, 1]
    
    corners = []
    for x, y in zip(xCorners, yCorners): 
        corners.append(wcsGetRaDecPair(tractWcs, x, y))
    
    return corners

# Understand the SkyMap of coadded images better 

## Return the name of all folders under certain directory

In [6]:
def getTractList(rootDir, filter, imgType='deepCoadd', toInt=True): 
    
    if rootDir[-1] is not '/': 
        rootDir += '/'
    tractDir = rootDir + imgType + '/' + filter + '/'
    
    if not os.path.exists(tractDir): 
        raise Exception("Can not find the directory: %s" % tractDir)
    else:
        tractStrs = [d for d in os.listdir(tractDir) if 
                     os.path.isdir(os.path.join(tractDir, d))]
        if toInt: 
            tractList = map(lambda x: int(x), tractStrs)
        else: 
            tractList = tractStrs
    
    return tractList

In [7]:
# Plot a polygon region
def plotListPoly(polyList, outPNG='polyList.png', outer=None, 
                 color=None, minX=None, minY=None, maxX=None,
                 maxY=None, xSize=20, ySize=16, dpi=120):

    """ 
    Right now, have to make sure that every Polygon in the list is 
    simple and valid TODO
    """
    
    """ Set up the color """
    BLUE = '#6699cc'
    GRAY = '#999999'
    ec = GRAY 
    if color is None: 
        fc = BLUE
    else:
        fc = color 

    fig = plt.figure(figsize=(xSize, ySize), dpi=dpi)
    ax = fig.add_subplot(111)
    
    if len(polyList) == 1: 
        partShow = PolygonPatch(polyList[0], fc='r', ec=GRAY,
                                alpha=0.8, zorder=1)
        ax.add_patch(partShow)
    else: 
        for poly in polyList:
            partShow = PolygonPatch(poly, fc=numpy.random.rand(3,1), 
                                    ec=GRAY, alpha=0.8, zorder=1)
            ax.add_patch(partShow)
        
    """ Draw the outline of the region """
    if outer is not None: 
        if outer.type is "Polygon": 
            bound = outer.boundary 
            if bound.type is "LineString": 
                x, y = bound.xy 
                ax.plot(x, y, c='k', lw=2.5)
        elif outer.type is "MultiPolygon":
            for oo in outer: 
                bound = oo.boundary
                if bound.type is "LineString":
                    x, y = bound.xy
                    ax.plot(x, y, c='k', lw=3.0)
                 
    if (minX is None) or (minY is None) or (maxX is None) or (maxY is None):
        ax.margins(0.02, 0.02, tight=True)
    else: 
        raRange  = [(minX-0.1), (maxX+0.1)]
        decRange = [(minY-0.1), (maxY+0.1)]
        ax.set_xlim(*raRange)
        ax.set_ylim(*decRange)

    ax.set_xlabel(r'RA (deg)',  fontsize=25)
    ax.set_ylabel(r'DEC (deg)', fontsize=25)

    fontsize = 16
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)

    ax.minorticks_on()
    plt.tick_params(which='major', width=2.0, length=8.0, labelsize=20)
    plt.tick_params(which='minor', width=1.8, length=6.0)

    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2.5)

    ax.grid(alpha=0.6, color='k', linewidth=1.5)
    
    fig.subplots_adjust(bottom=0.1, left=0.1,
                        top=0.98, right=0.98)

    fig.savefig(outPNG, dpi=100)
    plt.close(fig)

In [8]:
def coaddTractGetCorners(skyMap, tractId): 
    
    """ Try to get the right tract information """
    numTract = len(skyMap)
    if tractId > numTract: 
        raise Exception("The tractId is not correct: %d" % tractId)
    
    tractInfo = skyMap[tractId]
    if tractInfo.getId() != tractId: 
        raise Exception("Something wrong with the SkyMap: %d - %d" % (tractId, 
                                                                      tractInfo.getId()))
    else: 
        corners = tractInfo.getVertexList()
        cornerRaDec = []
        for corner in corners: 
            cornerRaDec.append((corner[0].asDegrees(), 
                                corner[1].asDegrees()))
    
    """ Form a Polygon using the cornerRaDec """
    cornerPoly = Polygon(cornerRaDec)
    if not cornerPoly.boundary.is_simple: 
        raise Exception("Something is wrong with the Polygon: %d" % tractId)
    
    """ Return this Polygon """
    return cornerPoly
    
    
def coaddTractShape(rootDir, filter, verbose=True, prefix='hsc_tract', 
                    savePNG=True, xSize=18, ySize=16): 
    
    """ Prefix of the output file """
    preFile1 = prefix + '_' + filter + '_tract'
    
    """ Get the skymap """
    butler = dafPersist.Butler(rootDir)
    skyMap = butler.get("deepCoadd_skyMap", immediate=True)
    
    """ Get the list of tract IDs """
    tractList = getTractList(rootDir, filter, imgType='deepCoadd', 
                             toInt=True)
    nTracts = len(tractList)
    if verbose: 
        print "### Will deal with %d tracts in total" % nTracts
        
    polyList = []
    for tractId in tractList: 
        if verbose: 
            print "### Deal with tract: %d" % tractId 
            
        """ Get the Polygon for the tract """
        tractPoly = coaddTractGetCorners(skyMap, tractId)
        
        preFile2 = preFile1  + '_' + str(tractId)
        """ Save a .WKB file """
        tractWkb = preFile2 + '.wkb'
        coaddPS.polySaveWkb(tractPoly, tractWkb)
        """ Save a .REG file """
        tractReg = preFile2 + '.reg'
        coaddND.polySaveReg(tractPoly, tractReg, color='green')        
        """ Append to the list """
        polyList.append(tractPoly)
        
    if nTracts > 1:
        """ Get the cascaded_union of all the Tracts """
        combPoly = cascaded_union(polyList)
        """ Save a combined .WKB file """
        coaddPS.polySaveWkb(combPoly, preFile1 + '_all.wkb')
        """ Save a combined .REG file """
        coaddND.polySaveReg(combPoly, preFile1 + '_all.reg', color='blue')
        """ Get the bounds of the combined region """
        minX, minY, maxX, maxY = combPoly.bounds
        
        """ It's possible that the combined Poly is Multi-part """
        if combPoly.type is "MultiPolygon": 
            combParts = combPoly.geoms[:]
            nParts = len(combParts)
            for ii in range(nParts): 
                combPart = combParts[ii]
                min1, min2, max1, max2 = combPart.bounds
                """ Save .wkb and .reg file for each part """
                coaddPS.polySaveWkb(combPart, preFile1 + '_part_' + str(ii+1) + '.wkb')
                coaddND.polySaveReg(combPart, preFile1 + '_part_' + str(ii+1) + '.reg', 
                                    color='blue')
                """ Make a plot """
                plotListPoly(polyList, outer=combPart, 
                             outPNG=preFile1 + '_part_' + str(ii+1) + '.png', 
                             minX=min1, minY=min2, maxX=max1, maxY=max2, 
                             xSize=16, ySize=14)
    else: 
        combPoly = polyList[0]
        minX, minY, maxX, maxY = combPoly.bounds
        coaddPS.polySaveWkb(combPoly, preFile1 + '_all.wkb')
        coaddND.polySaveReg(combPoly, preFile1 + '_all.reg', color='blue')
        polyList = [combPoly]
    
    """ Save a PNG file """
    if savePNG: 
        pngFile = preFile1 + '_all.png'
        plotListPoly(polyList, outPNG=pngFile, outer=combPoly, 
                     minX=minX, minY=minY, maxX=maxX, maxY=maxY, 
                     xSize=xSize, ySize=ySize) 
        
    return combPoly

In [9]:
filters  = ['HSC-G', 'HSC-R', 'HSC-I', 'HSC-Z', 'HSC-Y']
rootDir = '/lustre/Subaru/SSP/rerun/yasuda/SSP3.4.1_20141224/'
prefix  = 'ssp341_wide'

#for filter in filters: 
#    coaddTractShape(rootDir, filter, verbose=False, prefix=prefix, 
#                    savePNG=True, xSize=40, ySize=12)
    
#rootDir = '/lustre/Subaru/SSP/rerun/yasuda/SSP3.6.1_20150325/'
#prefix  = 'ssp361_widecos'

#for filter in filters: 
#    coaddTractShape(rootDir, filter, verbose=False, prefix=prefix, 
#                    savePNG=True, xSize=30, ySize=14)
    
#rootDir = '/lustre/Subaru/SSP/rerun/yasuda/SSP3.4.1_cosmos_setWeight/'
#prefix  = 'ssp341_cosmos'

#for filter in filters: 
#    coaddTractShape(rootDir, filter, verbose=False, prefix=prefix, 
#                    savePNG=True, xSize=12, ySize=12)

## Get the Tracts Lists

In [12]:
root1 = '/lustre/Subaru/SSP/rerun/yasuda/SSP3.4.1_20141224/'
root2 = '/lustre/Subaru/SSP/rerun/yasuda/SSP3.6.1_20150325/' 
root3 = '/lustre/Subaru/SSP/rerun/yasuda/SSP3.4.1_cosmos_setWeight/'
rootList = [root1, root2, root3]
prefixList = ['ssp341_wide', 'ssp361_widecos', 'ssp341_cosmos']

filters  = ['HSC-G', 'HSC-R', 'HSC-I', 'HSC-Z', 'HSC-Y']

for root, prefix in zip(rootList, prefixList): 
    for filter in filters: 
        preFile = prefix + '_' + filter + '_tract.lis'
        print "### Prepare %s" % preFile 
        tractFile = open(preFile, 'w')
        
        tractStrs = getTractList(root, filter, imgType='deepCoadd', toInt=False)
        
        for ss in tractStrs: 
            tractFile.write(ss + '\n')
        
        tractFile.close()
        print "### Done !"

### Prepare ssp341_wide_HSC-G_tract.lis
### Done !
### Prepare ssp341_wide_HSC-R_tract.lis
### Done !
### Prepare ssp341_wide_HSC-I_tract.lis
### Done !
### Prepare ssp341_wide_HSC-Z_tract.lis
### Done !
### Prepare ssp341_wide_HSC-Y_tract.lis
### Done !
### Prepare ssp361_widecos_HSC-G_tract.lis
### Done !
### Prepare ssp361_widecos_HSC-R_tract.lis
### Done !
### Prepare ssp361_widecos_HSC-I_tract.lis
### Done !
### Prepare ssp361_widecos_HSC-Z_tract.lis
### Done !
### Prepare ssp361_widecos_HSC-Y_tract.lis
### Done !
### Prepare ssp341_cosmos_HSC-G_tract.lis
### Done !
### Prepare ssp341_cosmos_HSC-R_tract.lis
### Done !
### Prepare ssp341_cosmos_HSC-I_tract.lis
### Done !
### Prepare ssp341_cosmos_HSC-Z_tract.lis
### Done !
### Prepare ssp341_cosmos_HSC-Y_tract.lis
### Done !


## Get information for the Patch

In [70]:
""" Get the skymap """
butler = dafPersist.Butler(rootDir)
skyMap = butler.get("deepCoadd_skyMap", immediate=True)

In [83]:
map = skyMap[9009]

In [75]:
patchInfo = map.getPatchInfo((2,2))

In [78]:
patchInfo.getInnerBBox()

Box2I(Point2I(8000, 8000), Extent2I(4000, 4000))

In [79]:
map.getPatchInnerDimensions()

Extent2I(4000, 4000)

In [81]:
patchInfo.getOuterBBox()

Box2I(Point2I(7900, 7900), Extent2I(4200, 4200))

In [82]:
patchInfo.getIndex()

(2, 2)